In [8]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance
from sklearn import preprocessing as pre
import plotly.express as px

<h2> Here we are getting the data organized based on a specified starting year </h2>


In [48]:
#THIS IS THE YEAR WE WANT TO START AND FINISH CALCULATING THE AVERAGE FROM
#min year is 1980 because the stats are pretty sketchy prior

start_year = 2010
end_year = 2020

In [49]:
all_players_df = pd.read_csv("csv_files/player_stats_"+str(start_year)+".csv")
all_players_df = all_players_df.drop_duplicates('NAME', keep='first')
all_players_df['YEAR']= start_year

for year in range(start_year+1, end_year+1):
    
    #store current year in df
    current_year_players_df = pd.read_csv("csv_files/player_stats_"+str(year)+".csv")

    #since some players were traded during each season their stats are split first showing TOT (total season)
    #and the following rows showing their stats specific for their teams
    #since the total is always first we can drop all duplicates following the first instance (ASSUMING NO PLAYERS HAVE THE SAME NAME)
    current_year_players_df = current_year_players_df.drop_duplicates('NAME', keep='first')
    current_year_players_df['YEAR'] = year
    #appending the current year to all previous years 
    all_players_df = all_players_df.append(current_year_players_df, ignore_index=True)
    
    

all_players_df
    

,NAME,TEAM,POS,AGE,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,YEAR
0,Arron Afflalo,DEN,SG,24,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
1,Alexis Ajinça,CHA,C,21,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
2,LaMarcus Aldridge,POR,PF,24,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
3,Joe Alexander,CHI,SF,23,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
4,Malik Allen,DEN,PF,31,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5354,Thaddeus Young,CHI,PF,31,64,16,24.9,4.2,9.4,.448,...,1.5,3.5,4.9,1.8,1.4,0.4,1.6,2.1,10.3,2020
5355,Trae Young,ATL,PG,21,60,60,35.3,9.1,20.8,.437,...,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6,2020
5356,Cody Zeller,CHO,C,27,58,39,23.1,4.3,8.3,.524,...,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1,2020
5357,Ante Žižić,CLE,C,23,22,0,10.0,1.9,3.3,.569,...,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4,2020


In [59]:
# Adding helper method
def make_per_36(df): 
    for index , player in df.iterrows():
        if player['MP'] == 0:
            continue
        else:
            multiplier = 36 / player['MP']

        df.loc[index,['MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']] *= multiplier
       
    
        
        if index % int(len(df)*0.25) == 0:
            print(str(index) + " players stats adjusted")
        
    return df
        

In [68]:
#HERE WE WILL SET CRITERIA FOR OUR DATA BASE AND GET RID OF NANs

#filling null values with 0
all_players_df = all_players_df.fillna(0)

######### UNCOMMENT IF YOU ONLY WANT TO LOOK AT CERTAIN POSITIONS #############
position = 'C'
players_df = all_players_df[all_players_df['POS'] == ' ' + position]

######### UNCOMMENT IF YOU ONLY WANT TO LOOK AT CERTAIN TEAM #############
# team = 'TOR'
# players_df = all_players_df[all_players_df['TEAM'] == ' ' + team]

######## UNCOMMENT TO PUT IN SOME PARAMETERS For Minutes Played #######3
players_df = all_players_df[all_players_df['MP'] >= 15]
players_df = players_df[players_df['G'] >= 20]


######### UNCOMMENT IF WE WANT ALL PLAYERS #############
# players_df = all_players_df

players_df = players_df.reset_index()

players_df=make_per_36(players_df)
players_df

0 players stats adjusted
849 players stats adjusted
1698 players stats adjusted
2547 players stats adjusted
3396 players stats adjusted


,index,NAME,TEAM,POS,AGE,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,YEAR
0,0,Arron Afflalo,DEN,SG,24,82,75,36.0,4.383764,9.431734,...,0.929889,3.188192,4.118081,2.258303,0.797048,0.531365,1.195572,3.586716,11.690037,2010
1,2,LaMarcus Aldridge,POR,PF,24,78,78,36.0,7.104000,14.400000,...,2.400000,5.376000,7.680000,2.016000,0.864000,0.576000,1.248000,2.880000,17.184000,2010
2,5,Ray Allen*,BOS,SG,34,80,80,36.0,5.931818,12.477273,...,0.613636,2.659091,3.272727,2.659091,0.818182,0.306818,1.636364,2.352273,16.670455,2010
3,6,Tony Allen,BOS,SG,28,54,8,36.0,5.236364,10.254545,...,2.181818,3.709091,5.890909,2.836364,2.400000,0.872727,2.618182,4.363636,13.309091,2010
4,7,Rafer Alston,TOT,PG,33,52,38,36.0,3.956044,11.340659,...,0.395604,2.901099,3.296703,4.483516,1.318681,0.263736,2.373626,2.637363,10.813187,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,5352,Delon Wright,DAL,SG,27,65,5,36.0,4.645161,9.788018,...,1.658986,4.976959,6.635945,5.640553,1.990783,0.497696,1.658986,2.156682,12.110599,2020
3394,5354,Thaddeus Young,CHI,PF,31,64,16,36.0,6.072289,13.590361,...,2.168675,5.060241,7.084337,2.602410,2.024096,0.578313,2.313253,3.036145,14.891566,2020
3395,5355,Trae Young,ATL,PG,21,60,60,36.0,9.280453,21.212465,...,0.509915,3.773371,4.385269,9.484419,1.121813,0.101983,4.895184,1.733711,30.186969,2020
3396,5356,Cody Zeller,CHO,C,27,58,39,36.0,6.701299,12.935065,...,4.363636,6.701299,11.064935,2.337662,1.090909,0.623377,2.025974,3.740260,17.298701,2020


<h2> First we will look at the age distrobusion of NBA Players </h2>

In [69]:
fig = px.histogram(players_df, x="AGE")
fig.update_layout(bargroupgap=0.2)
fig.show()

<h2> Now we are going to find the average stats for this pool of players based upon their age </h2>

In [70]:
columns = ['G','MP', '3P%', 'FTA', 'FT%','ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS' ]

df_by_age = pd.DataFrame(columns = ['AGE'] + columns )

#iterating through all ages
for age in range(19,41): 
    
    curr_dict = dict.fromkeys(columns)

    #extrating players that are only that age and the stats we can find the average of 
    curr_df = players_df[players_df['AGE'] == age]
    curr_df = curr_df[columns]
    
    #getting the mean 
    avg_player = curr_df.mean()    
    
    for col in columns:
        curr_dict[col] = avg_player[col]
    
    curr_dict['AGE'] = age
    df_by_age = df_by_age.append(curr_dict, ignore_index=True)
        
  
    
df_by_age

,AGE,G,MP,3P%,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,19.0,63.214286,36.0,0.280905,3.452434,0.711524,1.721120,4.853938,6.570292,2.702374,1.073512,1.007940,2.233642,3.337155,14.392423
1,20.0,67.043860,36.0,0.294360,3.484924,0.726553,1.817071,4.921409,6.731492,2.817831,1.164888,0.902586,2.163616,3.406573,14.668689
2,21.0,65.373563,36.0,0.267661,3.570007,0.735632,1.802133,4.903353,6.716543,3.051189,1.157634,0.817310,2.172458,3.261381,15.245705
3,22.0,64.750000,36.0,0.285716,3.478105,0.740237,1.761343,4.958975,6.718344,3.082924,1.157117,0.811705,2.076054,3.200151,15.331974
4,23.0,64.578073,36.0,0.288003,3.317661,0.746282,1.626401,4.716622,6.339098,3.180512,1.184556,0.745243,2.041080,3.113980,14.964659
5,24.0,64.235294,36.0,0.286056,3.405982,0.750935,1.655051,4.759210,6.404261,3.151619,1.166081,0.761624,1.962700,3.092064,15.131671
6,25.0,65.125000,36.0,0.298253,3.526151,0.755543,1.655166,4.786718,6.433531,3.179444,1.192180,0.776127,1.979586,3.041926,15.639573
7,26.0,64.376271,36.0,0.290136,3.510889,0.758895,1.627069,4.743162,6.367028,3.311047,1.144081,0.742339,1.996297,2.986480,15.572971
8,27.0,64.815972,36.0,0.285583,3.448434,0.771090,1.559545,4.836311,6.388198,3.398430,1.079985,0.701490,2.037369,2.944621,15.417006
9,28.0,64.866412,36.0,0.300576,3.356009,0.760084,1.525747,4.812573,6.333129,3.415247,1.113985,0.687155,2.011467,3.043139,15.388993


In [71]:
fig = px.bar(df_by_age, x='AGE', y='TOV')
fig.show()

In [65]:
scatter_df = df_by_age[['AGE','PTS','3P%','MP']]

fig = px.scatter_matrix(scatter_df)
fig.show()